# CSC380 Decision Tree Classifier Project

<div>
        Members: Sebastian Arana and Austin Baber
</div>

In [ ]:
import warnings
# Suppress warnings
warnings.filterwarnings("ignore")

### Imports
<div>We are using scikit-learn, numpy, pandas, matplotlib</div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,ConfusionMatrixDisplay

### Linear Boundary Flag
<p>Toggles linear vs nonlinear boundary</p>

In [ ]:
linear_bound = False